In [ ]:
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
#!pip install mysql
!pip3 install mysql-connector-python-rf
import mysql.connector
from pprint import pprint
import pymongo
import json
import numpy
import datetime
import certifi

In [ ]:
# local file system using CSV
data = pd.read_csv("products.csv")
data.head()

In [ ]:
for col in data.columns:
    print(col)

In [ ]:
# transforming existing data by deleting NA values

new_data = data.dropna()
print(new_data)

In [ ]:
# MongoDB
# declaring and assigning connection variables
mysql_args = {
    "uid" : "root",
    "pwd" : "1014810473",
    "hostname" : "localhost",
    "dbname" : "adventureworks_dw"
}

mongodb_args = {
    "user_name" : "uec6ct",
    "password" : "RNu3n7QuGWV1FOg3",
    "cluster_name" : "cluster0",
    "cluster_subnet" : "y4eygtf",
    "cluster_location" : "atlas", 
    "db_name" : "adventureworks_dw"
}

In [ ]:
# defining functions for getting data from and setting data into databases

def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [ ]:
# populating MongoDB with source data

client = get_mongo_client(**mongodb_args)

data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"purchase" : 'purchase.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files) 

In [ ]:
# extracting data from the source MongoDB into purchase dataframe

client = get_mongo_client(**mongodb_args)

query = {} 
collection = "purchase"

df_purchase = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)

In [ ]:
sql_dim_date = "SELECT date_key, full_date FROM adventureworks_dw.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date

In [ ]:
df_dim_DueDate = df_dim_date.rename(columns={"date_key" : "DueDate_key", "full_date" : "DueDate "})
df_DueDates.DueDate  = df_DueDates.DueDate.astype('datetime64[ns]').dt.date
df_DueDates = pd.merge(df_DueDates, df_dim_DueDate, on='DueDate', how='left')
df_DueDates.drop(['DueDate '], axis=1, inplace=True)

In [ ]:
# transformations to dataframes

df_DueDates.rename(columns={"id":"duedate_id"}, inplace=True)

df_DueDates.drop(['due_date'], axis=1, inplace=True)

df_DueDates.insert(0, "DueDate_key", range(1, df_DueDates.shape[0]+1))

In [ ]:
# loading transford dataframes into new data warehouse

dataframe = df_DueDates
table_name = 'dim_DueDates'
primary_key = 'DueDate_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)